# 07_03: More data formats with Pandas

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
df = pd.read_csv('Planets.csv', thousands=',', parse_dates=['FirstVisited'],
                 dtype={'Diameter': np.float64, 'MeanTemperature': np.float64,
                        'Rings': 'category', 'MagneticField': 'category'})
df.info()

In [ ]:
df.to_pickle('myplanets.pkl')

In [ ]:
pd.read_pickle('myplanets.pkl').head()

In [ ]:
df.to_feather('planets.feather')

In [ ]:
pd.read_feather('planets.feather').head()

In [ ]:
# also: df.to_parquet('planets.parquet')
#       pd.read_parquet('planets.parquet')

In [ ]:
pd.read_json('Planets-records.json').head()

In [ ]:
# df.to_json('Planets-records.json', orient='records')

In [ ]:
pd.read_json('Planets-index.json', orient='index')

In [ ]:
pd.read_json('Planets-index.json')

In [ ]:
pd.read_json('Planets-split.json', orient='split')

In [ ]:
# pd.read_html('https://en.wikipedia.org/wiki/Athletics_at_the_2024_Summer_Olympics')

In [ ]:
pd.read_html('Athletics_at_the_2024_Summer_Olympics.html')

In [ ]:
# select on attribute with attrs = {'id': ...}
pd.read_html('Athletics_at_the_2024_Summer_Olympics.html', match='Rank')[0]

In [ ]:
pd.read_html('Athletics_at_the_2024_Summer_Olympics.html', match='Rank', index_col=1)[0].tail()

In [ ]:
pd.read_html('Athletics_at_the_2024_Summer_Olympics.html', match='Rank', index_col=1, skiprows=[44])[0].tail()